In [1]:
using StatsPlots, Statistics, Distributions, Random

In [ ]:
# helper functions for the BEAM algorithm

function generate_midpoints(n_bins)
    quantiles = Array{Float64}(undef, n_bins)
    quantile_midpoints = Array{Float64}(undef, n_bins)

    #generate the quantile midpoints
    for i = 1:n_bins
        quantiles[i] = i/n_bins
        quantile_midpoints[i] = i/n_bins - (1/2)*(1/n_bins)
    end

    return vec(quantiles), vec(quantile_midpoints)
end

function add_distributions(list1, list2)
    sum_list = Array{Float64}(undef, length(list1)^2)
    
    list1 = repeat(list1, length(list1))
    list1 = sort(list1)
    list2 = repeat(list2, length(list2))
    
    for i in eachindex(sum_list)
        sum_list[i] = list1[i] + list2[i]
    end
    
    return vec(sort(sum_list))
end

function multiply_distributions(list1, list2)
    sum_list = Array{Float64}(undef, length(list1)^2)
    
    list1 = repeat(list1, length(list1))
    list1 = sort(list1)
    list2 = repeat(list2, length(list2))
    
    for i in eachindex(sum_list)
        sum_list[i] = list1[i] * list2[i]
    end
    
    return vec(sort(sum_list))
end

function re_bin(distribution, n_bins)
    rebinned = reshape(distribution, n_bins, n_bins)
    rebinned = mean(rebinned, dims = 1)
    
    return vec(rebinned)
end

function calculate_mean(RV, n_bins)
    return sum(RV .* 1/n_bins)
end

function calculate_variance(RV, calculated_mean, n_bins)
    return sum(RV.^2 * (1/n_bins)) - calculated_mean^2
end

function multiplicative_true_mean(μ, n_RVs)
    # this function assumes that all RVs are i.i.d.
    #    recommend editing for modularity later
    return μ^n_RVs
end

function multiplicative_true_variance(μ, σ, n_RVs)
    # this function assumes that all RVs are i.i.d.
    #    recommend editing for modularity later
    variance = 1
    mean_prod_square = true_mean(μ^2, n_RVs)
    
    for i = 1:n_RVs
        # compute the second moment of the product distribution
        variance = variance * (σ^2 + μ^2)
    end
    
    # compute the second central moment
    variance = variance - mean_prod_square
    return variance
end

function iid_exponential_BEAM(λ, n_RVs, n_bins)
    #Instantiate IID Exponential RVs
    β = 1/λ
    RV_list = Array{Distribution}(undef, n_RVs)
    RV_list = [Exponential(β) for i = 1:n_RVs]

    # define the list of quantiles and quantile midpoints
    quantiles, quantile_midpoints = generate_midpoints(n_bins)
    
    final_RV = first(RV_list)
    final_RV = quantile.(final_RV, quantile_midpoints)

    for i = 2:length(RV_list)
        temp_list = quantile.(RV_list[i], quantile_midpoints)

        final_RV = add_distributions(temp_list, final_RV)
        final_RV = re_bin(final_RV, n_bins) 
    end

    return final_RV
end

In [ ]:
# helper functions for the Monte-Carlo simulation
function cumulative_average(list::Array)
    cumulative_averages = Array{Float64}(undef, length(list))
    cumulative_averages[1] = list[1]
    
    for i in 2:length(list)
        cumulative_averages[i] = mean(list[1:(i-1)])
    end
    
    return cumulative_averages
end

function cumulative_stddev(list::Array, Corrected::Bool)
    cumulative_stddevs = Array{Float64}(undef, length(list))
    cumulative_stddevs[1] = 0
    
    for i in 2:length(list)
        cumulative_stddevs[i] = std(list[1:(i-1)], corrected = Corrected)
    end
    
    return cumulative_stddevs
end

function cumulative_var(list::Array, Corrected::Bool)
    cumulative_var = Array{Float64}(undef, length(list))
    cumulative_var[1] = 0
    
    for i in 2:length(list)
        cumulative_var[i] = var(list[1:(i-1)], corrected = Corrected)
    end
    
    return cumulative_var
end


# simple Monte Carlo sim to evaluate sums of iid exponentials
function iid_exponential_montecarlo(λ, n_RVs, n_reps, multiplication::Bool)
    β = 1/λ
    
    dist = Exponential(β)
    output = Array{Float64}(undef, n_reps)
    
    for i in eachindex(output)
        RVs = rand(dist, n_RVs)
        if multiplication
            output[i] = prod(RVs)
        else
            output[i] = sum(RVs)
        end
    end
    
    return output
end


# simple Monte Carlo sim to evaluate the sums of iid normals
function iid_normals_montecarlo(μ, σ, n_RVs, n_reps, multiplication::Bool)
    dist = Normal(μ, σ)
    output = Array{Float64}(undef, n_reps)
    
    for i in eachindex(output)
        RVs = rand(dist, n_RVs)
        if multiplication
            output[i] = prod(RVs)
        else
            output[i] = sum(RVs)
        end
    end
    
    return output
end


# simple Monte Carlo sim to evaluate sums of non-iid normals
function non_iid_normals_montecarlo(μ̄::Array, σ̄::Array, n_reps, multiplication::Bool)
    n_RVs = length(μ̄)
    
    # instantiate a list of Random Variables based on the list
    #    of means and standard deviations and reshape into a 
    #    column vector
    dist_list = map(x -> Normal(x[1], x[2]), zip(μ̄, σ̄))
    
    output = Array{Float64}(undef, n_reps)
    
    for i = 1:n_reps
        random_draws = map(x -> rand(x), dist_list)
        if multiplication
            output[i] = prod(random_draws)
        else
            output[i] = sum(random_draws)
        end
    end
    
    return output
end

In [62]:
n_bins = 4000
n_RVs = 10
μ = 6
σ = 3

RV_list = Array{Distribution}(undef, n_RVs)

for i = 1:n_RVs
#     μ = rand(5:10)
#     σ = rand(1:6)
    
    temp_RV = Normal(μ, σ)
    RV_list[i] = temp_RV

end

In [63]:
quantiles, quantile_boundaries = generate_midpoints(n_bins)

final_RV = pop!(RV_list)
final_RV = quantile.(final_RV, quantile_boundaries)

for RV in RV_list
    temp_list = quantile.(RV, quantile_boundaries)
    
    final_RV = multiply_distributions(temp_list, final_RV)
    final_RV = re_bin(final_RV, n_bins) 
end

In [64]:
calc_mean = calculate_mean(final_RV, n_bins)
calc_var = calculate_variance(final_RV, calc_mean, n_bins)

true_mu = multiplicative_true_mean(μ, n_RVs)
true_var = multiplicative_true_variance(μ, σ, n_RVs)

println("Expected Additive Mean is: ", true_mu)
println("Calculated Additive Mean is: ", calc_mean)

println("Expected Additive Variance is: ", true_var)
println("Calculated Additive Variance is: ", calc_var)

Expected Additive Mean is: 60466176
Calculated Additive Mean is: 6.046617599999996e7
Expected Additive Variance is: 30394470475952649
Calculated Additive Variance is: 2.9677594031188616e16


In [65]:
true_var/calc_var

1.0241554771593229

In [58]:
true_var > calc_var

true